In [2]:
# Plotting tools
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Other tools
from nltk.tokenize import RegexpTokenizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from gensim.models import CoherenceModel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 18.8 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.8/dist-packages/gensim/corpora/dictionary.py:11: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping, defaultdict
/usr/local/lib/python3.8/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [4]:
# Load in state of the union address data
import pandas as pd
# Read in data
url = "/content/state_ofthe_union_texts.csv"
df = pd.read_csv(url)

# Subset df to Great Depression/WWII era
GD_WWII = df[(df["Year"] >1928) & (df["Year"] < 1946)]

In [5]:
# Tokenize 

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
stop_words = stopwords.words('english')
stop_words.extend(['shall', 'applause', 'nation', 'year', 'must', 'world', 'people', 'america', 'american', 'us', 'congress','u', 'y'])
#stop_words.extend(['applause','year', 'u', 'y', 'shall'])
#stop_words.extend(['applause','year', 'u', 'y', 'per', 'cent', 'nof', 'nto', 'nthe', 'nand', 'nin', 'ni', 'nit', 'na'])
#stop_words.extend(['applause','year', 'u', 'y', 'per', 'cent', 'tonight', 'today'])

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

# compile sample documents into a list
# doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]
doc_set = GD_WWII['Text'].values.tolist()

# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in stop_words]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)


# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)

# filter extremes
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, 
                                           num_topics=3, 
                                           id2word = dictionary, 
                                           passes=100)

for idx, topic in ldamodel.print_topics():
  print("Topic: {} \nWords: {}".format(idx, topic ))
  print("\n")
# print(ldamodel.print_topics(num_topics=10, num_words=10))

/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

Topic: 0 
Words: 0.021*"democraci" + 0.010*"democrat" + 0.009*"1933" + 0.008*"billion" + 0.008*"purchas" + 0.007*"selfish" + 0.006*"accept" + 0.006*"domest" + 0.006*"thought" + 0.006*"abus"


Topic: 1 
Words: 0.026*"bank" + 0.019*"construct" + 0.013*"per" + 0.013*"depart" + 0.013*"expenditur" + 0.012*"credit" + 0.011*"depress" + 0.011*"commiss" + 0.009*"emerg" + 0.009*"fiscal"


Topic: 2 
Words: 0.028*"fight" + 0.023*"enemi" + 0.016*"victori" + 0.013*"job" + 0.012*"air" + 0.010*"pacif" + 0.010*"german" + 0.009*"japan" + 0.009*"china" + 0.009*"sea"




/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [6]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=texts, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.4378494001824678


In [7]:
# Compute Perplexity
print('\nPerplexity: ', ldamodel.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -6.225680389771599


/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [8]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
vis

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.157903 -0.026119       1        1  43.874239
2     -0.108761 -0.090709       2        1  28.190688
0     -0.049143  0.116829       3        1  27.935073, topic_info=           Term       Freq      Total Category  logprob  loglift
480       fight  59.000000  59.000000  Default  30.0000  30.0000
40         bank  86.000000  86.000000  Default  29.0000  29.0000
128       enemi  51.000000  51.000000  Default  28.0000  28.0000
554   democraci  48.000000  48.000000  Default  27.0000  27.0000
594     victori  33.000000  33.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
53        broad   8.806142  15.666960   Topic3  -5.4426   0.6992
585         tri   8.781620  15.496142   Topic3  -5.4454   0.7074
479   everywher   8.783275  15.535152   Topic3  -5.4452   0.7050
144  expenditur   9.863437  51.493912   Topic3  -5.3292  -0.3773
524        tell   8.488924  15.494045   Topic3  -5.4792   0.6736

[166 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
0         1  0.837149      1
0         2  0.173203      1
441       1  0.136157   1933
441       2  0.045386   1933
441       3  0.771555   1933
...     ...       ...    ...
386       1  0.136041  wider
386       3  0.816247  wider
595       2  0.936996    win
392       1  0.136068  youth
392       3  0.816409  youth

[287 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])